In [1]:
from IPython.core.debugger import set_trace
import unittest
tc = unittest.TestCase('__init__')

import numpy as np
# mathjax docs: https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference

# The Problem

Given $A \in \mathbb{R}^{mxn} \text{ and } b \in \mathbb{R}^n$ in the equation:

$Ax = b$
  
We want to solve for $x$

In [153]:
def generate_random_problem(size=(3, 3)):
    return np.random.random(size), np.random.random((size[0],1))

def get_example_problem():
    # example 6.1.2
    return (np.array([
        [4, -1, 2 ,-1],
        [2, 6, 3, -3],
        [1, 1, 5, 0],
        [1, -1, 4, 7]
    ], dtype=np.float64), 
    np.array([[-8, -20, -2, 4]], dtype=np.float64).T,
    np.array([[2, 2, 3, -7]], dtype=np.float64).T)

In [3]:
def test_correct_matrix_solution(A, b, x):
    return np.array_equal(np.dot(A, x), b)

def test_method(method_func):
    # example 6.1.2
    A_1 = np.array([
        [1, -1, 2 ,-1],
        [2, -2, 3, -3],
        [1, 1, 1, 0],
        [1, -1, 4, 3]
    ])
    b_1 = np.array([[-8, -20, -2, 4]]).T
    x_1 = np.array([2, 2, 3, -7]).T
    tc.assertTrue(np.array_equal(method_func(A_1, b_1), x_1))
    
    # random A & b
    A_2 = np.random.random((4,4))
    b_2 = np.random.random((4,1))
    x_2 = method_func(A_2, b_2)
    tc.assertTrue(np.array_equal(np.dot(A_2, x_2), b_2))

# Gausse Elimination

### 1. No Pivoting:

In [4]:
def swap_rows(m, row_index_1, row_index_2):
    cpy = m.copy()
    cpy[[row_index_1, row_index_2]] = cpy[[row_index_2, row_index_1]]
    return cpy

def swap_rows_if_needed(m, b, pivot_row, pivot_col):
        if m[pivot_row, pivot_col] > 0:
            return m, b
        
        nonzeros = m[pivot_row: m.shape[0], pivot_col].nonzero()
        if len(nonzeros[0]) > 0:
            next_nonzero_index = nonzeros[0][0] + pivot_row
            if next_nonzero_index != pivot_row:
                m = swap_rows(m, pivot_row, next_nonzero_index)
                b = swap_rows(b, pivot_row, next_nonzero_index)

        return m, b

def swap_rows_if_needed_test():
    A = np.random.random((4,4))
    b = np.random.random((4,1))
    A[1, 1] = 0
    A_real, b_real = swap_rows_if_needed(A,b, 1, 1)
    A_expected = swap_rows(A, 1, 2)
    b_expected = swap_rows(b, 1, 2)
    tc.assertTrue(np.array_equal(A_expected, A_real))
    tc.assertTrue(np.array_equal(b_expected, b_real))
    
    B = np.random.random((4,4))
    c = np.random.random((4,1))
    B_real, c = swap_rows_if_needed(B, c, 1, 1)
    B_expected = B
    tc.assertTrue(np.array_equal(B_expected, B_real))
    
swap_rows_if_needed_test()

In [5]:
def forward_elimination(A_input, b_input):
    def forward_elimination_recurse(A, b, pivot_index):
        pivot_row_index = A.shape[1] - A.shape[0] + pivot_index
        A, b = swap_rows_if_needed(A, b, pivot_row_index, pivot_index)
        pivot_row = A[pivot_row_index]
        
        start_row_index = pivot_row_index + 1

        for row_i in range(start_row_index, A.shape[0]):
            multiplier = np.abs(pivot_row[pivot_index]) *  (A[row_i][pivot_index] / pivot_row[pivot_index])
            A[row_i] -= (multiplier * pivot_row)
            b[row_i] -= (multiplier * b[pivot_row_index])
        
        if pivot_index >= A.shape[1]-1:
            return A, b
        else:
            return forward_elimination_recurse(A, b, pivot_index + 1)
    
    return forward_elimination_recurse(A_input, b_input, 0)

In [6]:
# A -> n x m matrix, b -> 1 x m matrix
def backward_substitution(A, B):
    '''
    There are only m variables to solve for in an n x m matrix,
    so we can just solve the m x m submatrix. If m > n, we don't
    have enough equations to solve the system.

    x = output    
    n = # of rows
    k = # of columns
    '''
    x = np.zeros((B.shape[0],))
    n = A.shape[0]
    k = A.shape[1]
    
    # first iteration 
    x[k-1] = B[k-1, 0] / A[n-1,k-1]
    A[0: n-1,k-1] *= x[k-1]
    B[k-1] *= x[k-1]

    for i in reversed(range(0, k-1)):
        found_variables_sum = np.sum(A[i, i+1:n])
        x[i] = (B[i, 0] - found_variables_sum) / A[i, i]
        A[:i, i] *= x[i]

    if n > k:
        A[0: n-k] *= x
    
    return x[::-1]

In [7]:
def gauss_no_pivot(A, b):
    A, b = forward_elimination(A, b)
    return backward_substitution(A, b)

In [8]:
A, b, x = get_example_problem()

p_x = gauss_no_pivot(A, b)
np.array_equal(x.T[0], p_x)

True

In [9]:
A, b, x = get_example_problem() 
x @ np.array([[1,2,3,4]])

array([[  2.,   4.,   6.,   8.],
       [  2.,   4.,   6.,   8.],
       [  3.,   6.,   9.,  12.],
       [ -7., -14., -21., -28.]])

### 2. Partial Pivoting

In [10]:
def gauss_partial_pivot(A, b):
    pass

In [148]:
A, b, x = get_example_problem()

(2,)

# Jacobi Iterative Method

In [154]:
def jacobi_iterative(A, b):
    D = np.diagflat(np.diag(A))
    R = A - D
    L = np.tril(R)
    U = np.triu(R)
    inverse_D = np.diagflat(np.reciprocal(np.diag(A)))
    prev_x = np.random.random(b.shape)
    converg_thresh = .1
    error = np.inf
    
    while error > converg_thresh:
        x = np.zeros(b.shape)
        for i in range(b.shape[0]):
            x[i] = b[i] - np.sum(A[i] * prev_x.T)
        
        error = np.linalg.norm(x - prev_x) / np.linalg.norm(x)
        prev_x = x
    
    return x

In [155]:
# http://mathonline.wikidot.com/applying-the-jacobi-iteration-method
A, b, x = get_example_problem()
print(jacobi_iterative(A, b))
print(" ")
print("ANSWER")
print(x)

[[-1.55525125e+153]
 [-1.08327362e+154]
 [-2.26165369e+152]
 [ 1.45103637e+154]]
 
ANSWER
[[ 2.]
 [ 2.]
 [ 3.]
 [-7.]]


/usr/local/Cellar/python3/3.5.0/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
